In [ ]:
import fitz  # PyMuPDF
import pandas as pd
from langchain_cohere import CohereEmbeddings

In [ ]:
# !pip install fitz

### Preprocessing Pdf

In [ ]:
def extract_questions_from_pdf(pdf_path):
    document = fitz.open(pdf_path)
    questions = []
    for page_num in range(document.page_count):
        page = document.load_page(page_num)
        text = page.get_text()

        # Split text into lines
        lines = text.split('__') # text.split('\n') was cuttign off the text for questions that spanned multiple lines
        lines = [line.replace('\n','') for line in lines]
        # Extract lines that are questions (beginning with a bullet point)
        for line in lines:
            if line.strip().startswith('•'):
                questions.append(line.strip()[1:].strip())
    return questions

In [ ]:
pdf_path = '/home/ethel/Documents/SOSE 2024/DATA SCIENCE/project/Uni Supply Chain/SEC Questionaire 3.pdf'
questions = extract_questions_from_pdf(pdf_path)
questions_df = pd.DataFrame(questions, columns=["Questions"])
questions_df['Questions'].to_list()

In [ ]:
embeddings_model = CohereEmbeddings(cohere_api_key="yU3vQ4wcV8gbMCvsHngfAa6OfVmnaMQy1YOpVwqF")
embeddings_pdf = embeddings_model.embed_documents(questions_df['Questions'].to_list())
embeddings_pdf[0]

### Preprocessing Images

In [ ]:
import pytesseract #on linux, you need to install tesseract-ocr
from PIL import Image

# Paths to the images
image_paths = [
    '/home/ethel/Documents/SOSE 2024/DATA SCIENCE/project/Uni Supply Chain/Sec_Questionaire1.png',
    '/home/ethel/Documents/SOSE 2024/DATA SCIENCE/project/Uni Supply Chain/Sec_Questionaire2.png',
    '/home/ethel/Documents/SOSE 2024/DATA SCIENCE/project/Uni Supply Chain/Sec_Questionaire3.png'
]

# Function to extract text from an image
def extract_text_from_image(image_path):
    image = Image.open(image_path)
    text = pytesseract.image_to_string(image, lang='deu', config='--psm 6')
    return text

# Extract text from each image
extracted_texts = [extract_text_from_image(image_path) for image_path in image_paths]
extracted_texts

In [ ]:
embeddings_images = embeddings_model.embed_documents(extracted_texts)
embeddings_images

### Preprocessing Excel

In [ ]:

file_path = '/home/ethel/Documents/SOSE 2024/DATA SCIENCE/project/Uni Supply Chain/Security_Questionnaire_Template.V1.5.xlsx'
excel_data = pd.ExcelFile(file_path)

# Load the sheet that contains the questions and answers
questions_sheet = pd.read_excel(file_path, sheet_name='Questionnaire',header=3)

# Extract the relevant columns for questions and their corresponding answers
questions_answers = questions_sheet[['Questions', "Supplier's Answer"]]

# Drop rows where questions or answers are not present
questions_answers = questions_answers.dropna(subset=['Questions', "Supplier's Answer"])
questions_answers

In [ ]:
excel_questions_embeddings = embeddings_model.embed_documents(questions_answers['Questions'].to_list())
excel_answers_embeddings = embeddings_model.embed_documents(questions_answers["Supplier's Answer"].to_list())
excel_questions_embeddings